# Wysłanie współrzędnych i otrzymanie adresu

In [ ]:
import geopandas as gpd
import requests
from shapely.geometry import Point, LineString, Polygon
import os
import rasterio
import numpy as np
import pandas as pd
from rasterio.mask import mask
from pathlib import Path

# Pozyskiwanie adresów przez API (pominąć)  
Z uwagi na ograniczenia w pozyskiwaniu danych w ten sposób porzucono wykorzystywanie API na rzecz gotowych skorowidzów. Kod w tej sekcji jest jedynie do demonstracji.

In [ ]:
def points_from_vector_layer(path_to_vector,density=250,target_crs="EPSG:2180"):
    """
    Generuje punkty zapytań (shapely Point) na podstawie warstwy wektorowej
    """
    gdf = gpd.read_file(path_to_vector)

    if gdf.crs is None or gdf.crs.to_string() != target_crs:
        gdf = gdf.to_crs(target_crs)

    points = []

    for geom in gdf.geometry:
        if geom is None or geom.is_empty:
            continue
        if geom.geom_type == "Point":
            points.append(geom)
        elif geom.geom_type in ["LineString", "MultiLineString"]:
            lines = [geom] if geom.geom_type == "LineString" else geom.geoms

            for line in lines:
                length = line.length
                n = max(int(length / density), 1)

                for i in range(n + 1):
                    point = line.interpolate(i / n, normalized=True)
                    points.append(point)

        if geom.geom_type in ["Polygon", "MultiPolygon"]:
            polygon = geom if geom.geom_type == "Polygon" else unary_union(geom)

            cx, cy = polygon.centroid.coords[0]
            minx, miny, maxx, maxy = polygon.bounds

            x_coords = np.arange(cx, maxx + density, density)
            x_coords = np.concatenate([x_coords, np.arange(cx - density, minx - density, -density)])

            y_coords = np.arange(cy, maxy + density, density)
            y_coords = np.concatenate([y_coords, np.arange(cy - density, miny - density, -density)])

            for x in x_coords:
                for y in y_coords:
                    p = Point(x, y)
                    if polygon.covers(p):
                        points.append(p)

    return points


In [ ]:
import requests
import datetime
import re
from shapely.geometry import Point
from typing import List, Dict, Any
from urllib3.exceptions import InsecureRequestWarning
import warnings
import time

warnings.filterwarnings("ignore", category=InsecureRequestWarning)
LAS_EVRF_WMS_URL = "https://opendata.geoportal.gov.pl/NumDaneWys/WMS/LAS_EVRF_WMS"

expr = re.compile(r"\{.*?\}")

def get_feature_info(params, url, retries=3, timeout=20, sleep_s=0.3):
    for attempt in range(retries):
        try:
            r = requests.get(url, params=params, verify=False, timeout=timeout)
            r.raise_for_status()
            return r.text
        except requests.exceptions.ReadTimeout:
            print('wewnetrzny timeout')
            if attempt == retries - 1:
                raise
            time.sleep(sleep_s)
    r = requests.get(url, params=params, verify=False, timeout=20)
    r.raise_for_status()
    return r.text

def remove_duplicates_from_list_of_dicts(dict_list):
    seen = set()
    unique_dict_list = []
    for d in dict_list:
        fset = frozenset(d.items())
        if fset not in seen:
            seen.add(fset)
            unique_dict_list.append(d)
    return unique_dict_list

# Wersja z opóźniaczem
def getLasListbyPoint1992(point, LAS_EVRF_WMS_URL):
    x, y = point.x, point.y
    layers = ['SkorowidzeLIDAR2022iStarsze', 'SkorowidzeLIDAR2023', 'SkorowidzeLIDAR2024', 'SkorowidzeLIDAR2025']

    params = {
        'SERVICE': 'WMS',
        'REQUEST': 'GetFeatureInfo',
        'VERSION': '1.3.0',
        'LAYERS': ','.join(layers),
        'QUERY_LAYERS': ','.join(layers),
        'CRS': 'EPSG:2180',
        'BBOX': f'{y-50},{x-50},{y+50},{x+50}',
        'WIDTH': 101,
        'HEIGHT': 101,
        'I': 50,
        'J': 50,
        'INFO_FORMAT': 'text/html',
        'FORMAT': 'image/png'
    }

    html = get_feature_info(params, LAS_EVRF_WMS_URL)

    # parsowanie HTML -> lista kafli
    elems = expr.findall(html)
    sub_list = []
    for e in elems:
        attrs = {}
        parts = e.strip("{}").split(',')
        for p in parts:
            k, v = p.split(':', 1)
            attrs[k.strip('" ')] = v.strip('" ')
        sub_list.append(attrs)

    # konwersja typów na float/int/date tak jak w wtyczce
    for elem in sub_list:
        if 'aktualnosc' in elem:
            elem['aktualnosc'] = datetime.datetime.strptime(elem['aktualnosc'], '%Y-%m-%d').date()
        for key in ['charakterystykaPrzestrzenna', 'bladSredniWysokosci', 'bladSredniPolozenia']:
            if key in elem:
                elem[key] = float(elem[key].split()[0]) if key=='charakterystykaPrzestrzenna' else float(elem[key])
        if 'aktualnoscRok' in elem:
            elem['aktualnoscRok'] = int(elem['aktualnoscRok'])

    return remove_duplicates_from_list_of_dicts(sub_list)


In [ ]:
# Wersja bez opóźniacza
def getLasListbyPoint1992(point):
    x, y = point.x, point.y
    layers = ['SkorowidzeLIDAR2022iStarsze', 'SkorowidzeLIDAR2023', 
              'SkorowidzeLIDAR2024', 'SkorowidzeLIDAR2025']

    params = {
        'SERVICE': 'WMS',
        'REQUEST': 'GetFeatureInfo',
        'VERSION': '1.3.0',
        'LAYERS': ','.join(layers),
        'QUERY_LAYERS': ','.join(layers),
        'CRS': 'EPSG:2180',
        'BBOX': f'{y-50},{x-50},{y+50},{x+50}',
        'WIDTH': 101,
        'HEIGHT': 101,
        'I': 50,
        'J': 50,
        'INFO_FORMAT': 'text/html',
        'FORMAT': 'image/png'
    }

    resp = requests.get(params=params, verify=False, timeout=20)
    resp.raise_for_status()
    html = resp.text

    # parsowanie HTML -> lista kafli
    elems = expr.findall(html)
    sub_list = []
    for e in elems:
        attrs = {}
        parts = e.strip("{}").split(',')
        for p in parts:
            k, v = p.split(':', 1)
            attrs[k.strip('" ')] = v.strip('" ')
        sub_list.append(attrs)

    # konwersja typów na float/int/date tak jak w wtyczce
    for elem in sub_list:
        if 'aktualnosc' in elem:
            elem['aktualnosc'] = datetime.datetime.strptime(elem['aktualnosc'], '%Y-%m-%d').date()
        for key in ['charakterystykaPrzestrzenna', 'bladSredniWysokosci', 'bladSredniPolozenia']:
            if key in elem:
                elem[key] = float(elem[key].split()[0]) if key=='charakterystykaPrzestrzenna' else float(elem[key])
        if 'aktualnoscRok' in elem:
            elem['aktualnoscRok'] = int(elem['aktualnoscRok'])

    return remove_duplicates_from_list_of_dicts(sub_list)

# Odczytywanie adresów z lokalnie pobranych skorowidzów

In [2]:
# pozyskanie danych na skorowidze
def import_skorowidze(folder, target_crs="EPSG:2180"):
    gdfs = []

    for gml_path in Path(folder).glob("*.gml"):
        gdf = gpd.read_file(gml_path)

        # CRS
        if gdf.crs is None or gdf.crs.to_string() != target_crs:
            gdf = gdf.to_crs(target_crs)

        # standaryzacja nazw kolumn (na wszelki wypadek)
        rename_map = {
            "godlo": "godlo",
            "akt_rok": "akt_rok",
            "url_do_pobrania": "url"
        }

        gdf = gdf.rename(columns=rename_map)

        # zostawiamy tylko to co potrzebne
        gdf = gdf[["godlo", "akt_rok", "url", "geometry"]]


        gdfs.append(gdf)

    if not gdfs:
        raise ValueError("Nie znaleziono żadnych plików GML w folderze")

    out_gdf = gpd.GeoDataFrame(
        pd.concat(gdfs, ignore_index=True),
        crs=target_crs
    )
    out_gdf["filename"] = out_gdf["url"].apply(lambda u: u.split("/")[-1])

    return out_gdf
    
def filtruj_najnowsze(dane):
    dane = dane.sort_values("akt_rok", ascending=False).drop_duplicates(subset="godlo", keep="first").reset_index(drop=True)
    return dane

# usuwanie rekordów poza powierzchniami
def filtruj_po_polishp(skorowidze, shp):
    if skorowidze.crs != shp.crs:
        shp = shp.to_crs(skorowidze.crs)
    aoi_union = shp.geometry.union_all()

    mask = skorowidze.geometry.intersects(aoi_union)

    return skorowidze[mask].reset_index(drop=True)

# Sam mechanizm pobrania kafla na podstawie adresu

In [3]:
def pobierz_kafla(las_url, file_path):
    # wyciągamy nazwę pliku z URL
    file_name = las_url.split('/')[-1]
    file_path = os.path.join(file_path, file_name)

    #print(f"Pobieram {las_url} ...")
    try:
        with requests.get(las_url, stream=True) as r:
            r.raise_for_status()
            with open(file_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        #print(f"Pobrano plik do: {file_path}")
    except requests.exceptions.HTTPError as e:
        print(f"Błąd HTTP: {e}")
    except requests.exceptions.ConnectionError:
        print("Błąd połączenia")
    except Exception as e:
        print(f"Nieoczekiwany błąd: {e}")

In [4]:
# możliwe, że na ten moment to zbędne
def unique_las_items(items):
    by_godlo = {}

    for item in items:
        godlo = item.get('godlo')
        rok = item.get('aktualnoscRok', -1)

        if godlo not in by_godlo or rok > by_godlo[godlo].get('aktualnoscRok', -1):
            by_godlo[godlo] = item

    # dodatkowo usuwamy duplikaty po URL (na wypadek błędnych danych)
    seen_urls = set()
    out = []

    for item in by_godlo.values():
        url = item.get('url')
        if url not in seen_urls:
            seen_urls.add(url)
            out.append(item)

    return out

# <font color="orange">Proces</font>
Uruchamianie utworzonych funkcji i rozpoczynanie procesu pobierania danych. Przed uruchomieniem procesu wprowadzić poprawne ścieżki dla plików

In [ ]:
# zmienne
output_path = 'ścieżka dla danych wyjściowych'
shp_path = 'ścieżka dla pliku shp + nazwa pliku .shp'
skorowidze_path =  'Scieżka folderu ze skorowidzami'

## Pobranie skoroszytów

In [7]:
s_dane = import_skorowidze(skorowidze_path)
s_dane = filtruj_najnowsze(s_dane)
len(s_dane)

40586

In [11]:
# sprawdzenie położenia kafli wykonując warstwę shp
s_dane.to_file(r'D:\Studia Big Data\DYPLOMOWA\shp\kafle_kluczbork.shp')

In [8]:
# filtrowanie po terenach leśnych
shp = gpd.read_file(shp_path)
s_dane = filtruj_po_polishp(s_dane, shp)
len(s_dane)

333

In [ ]:
def zredukuj_pobrane(download_dir):
    download_dir = Path(download_dir)
    pobrane_pliki = {p.name for p in download_dir.glob("*.laz")}
    do_pobrania = s_dane[~s_dane["filename"].isin(pobrane_pliki)].copy()
    return do_pobrania

In [ ]:
# usuwanie z listy pobrane kafle jeśli proces został przerwany w trakcie
s_dane = zredukuj_pobrane(output_path)

## Pobranie kafli

In [13]:
from tqdm import tqdm
import time
for _, r in tqdm(s_dane.iterrows(), total=len(s_dane), desc="Pobieranie..."):
    pobierz_kafla(r["url"], output_path)


# <font color="orange">Mechanizm importowania plików NMT</font>
Pobieranie kafli nmt z Geoportalu w formacie .asc

In [ ]:
skoroszyt_nmt = r'ścieżka dla skoroszytów + nazwa danego skoroszytu .gml'
output_nmt = r'ścieżka zapisu nmt'

In [3]:
def read_skorowidz_nmt(gml_path, target_crs="EPSG:2180"):
    gdf = gpd.read_file(gml_path)

    # CRS (na wszelki wypadek)
    if gdf.crs is None:
        gdf = gdf.set_crs(target_crs)
    elif gdf.crs.to_string() != target_crs:
        gdf = gdf.to_crs(target_crs)

    cols = [
        "godlo",
        "url_do_pobrania",
        "format",
        "char_przestrz",
        "akt_rok",
        "geometry",
    ]

    return gdf[cols]

In [19]:
def pobierz_nmt_asc(url, out_dir, overwrite=False, timeout=60):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    file_name = url.split("/")[-1]
    out_path = out_dir / file_name

    if out_path.exists() and not overwrite:
        print(f"✔️ Już istnieje: {out_path.name}")
        return out_path

    print(f"⬇️ Pobieram: {file_name}")

    try:
        with requests.get(url, stream=True, timeout=timeout) as r:
            r.raise_for_status()

            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=1024 * 1024):
                    if chunk:
                        f.write(chunk)

        print(f"✅ Pobrano: {out_path}")
        return out_path

    except requests.exceptions.RequestException as e:
        print(f"❌ Błąd pobierania {url}\n{e}")
        return None


In [ ]:
# zmienne 
shp_rdlp_path = r'ścieżka dla pliku + nazwa pliku .shp'

In [ ]:
shp_rdlp = gpd.read_file(shp_rdlp_path)
dane_nmt = filtruj_po_polishp(dane_nmt, shp_rdlp)

In [22]:
for i, layer in dane_nmt.iterrows():
    pobierz_nmt_asc(layer['url_do_pobrania'], output_nmt)

⬇️ Pobieram: 77532_1356600_M-34-52-D-c-1-1.asc
✅ Pobrano: E:\kafle_las\NMT\77532_1356600_M-34-52-D-c-1-1.asc
⬇️ Pobieram: 77532_1356613_M-34-52-D-c-4-2.asc
✅ Pobrano: E:\kafle_las\NMT\77532_1356613_M-34-52-D-c-4-2.asc
⬇️ Pobieram: 77532_1356612_M-34-52-D-c-4-1.asc
✅ Pobrano: E:\kafle_las\NMT\77532_1356612_M-34-52-D-c-4-1.asc
⬇️ Pobieram: 77532_1356572_M-34-52-C-b-4-4.asc
✅ Pobrano: E:\kafle_las\NMT\77532_1356572_M-34-52-C-b-4-4.asc
⬇️ Pobieram: 77532_1356577_M-34-52-C-c-3-1.asc
✅ Pobrano: E:\kafle_las\NMT\77532_1356577_M-34-52-C-c-3-1.asc
⬇️ Pobieram: 77532_1356578_M-34-52-C-c-3-2.asc
✅ Pobrano: E:\kafle_las\NMT\77532_1356578_M-34-52-C-c-3-2.asc
⬇️ Pobieram: 77532_1356610_M-34-52-D-c-3-3.asc
✅ Pobrano: E:\kafle_las\NMT\77532_1356610_M-34-52-D-c-3-3.asc
⬇️ Pobieram: 77532_1356595_M-34-52-C-d-4-1.asc
✅ Pobrano: E:\kafle_las\NMT\77532_1356595_M-34-52-C-d-4-1.asc
⬇️ Pobieram: 77532_1356585_M-34-52-C-d-1-3.asc
✅ Pobrano: E:\kafle_las\NMT\77532_1356585_M-34-52-C-d-1-3.asc
⬇️ Pobieram: 77532_

KeyboardInterrupt: 

In [28]:
len(dane_nmt)

4283